<a href="https://colab.research.google.com/github/khizon/NBA_points/blob/main/NBA_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Using NBA's Stats API to collect shot data**

In [ ]:
!pip install nba_api --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.9/254.9 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 56.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.2 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.


In [ ]:
# Import packages
from nba_api.stats.endpoints import shotchartdetail
import json
import requests
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import time

plt.style.use('seaborn-darkgrid')

In [ ]:
# Load teams file
teams = pd.DataFrame(json.loads(requests.get('https://raw.githubusercontent.com/bttmly/nba/master/data/teams.json').text))
# Load players file
players = pd.DataFrame(json.loads(requests.get('https://raw.githubusercontent.com/bttmly/nba/master/data/players.json').text))

In [ ]:
# Get team ID based on team name
def get_team_id(team_abbreviation):
  try:
    return teams.loc[teams.abbreviation==team_abbreviation, 'teamId'].values[0]
  except:
    print(f'{team_abbreviation} is not an NBA Team')
    return -1
  
# Get player ID based on player name
def get_player_id(first, last):
  try:
    return players.loc[(players.firstName == first) & (players.lastName == last), 'playerId'].values[0]
  except:
    print(f'{first} {last} is not an NBA Player')
    return -1

# JSON to Df
def json_to_df(shot_json):
  # Load data into a Python dictionary
  shot_data = json.loads(shot_json.get_json())
  # Get the relevant data from our dictionary
  relevant_data = shot_data['resultSets'][0]
  # Get the headers and row data
  headers = relevant_data['headers']
  rows = relevant_data['rowSet']
  # Create pandas DataFrame
  df = pd.DataFrame(rows)
  df.columns = headers
  return df

In [ ]:
season = 2003

while season < 2023:
  if (season < 2010) | ((season >= 2014) & (season < 2018)):
    team = 'CLE'
  elif ((season>=2010) & (season<2014)):
    team = 'MIA'
  else:
    team = 'LAL'
  f_season = f'{season}-{(season+1)%100:02d}'
  print(f'{f_season} {team}')
  # Create JSON request
  shot_json = shotchartdetail.ShotChartDetail(
              team_id = get_team_id(team),
              player_id = get_player_id('LeBron', 'James'),
              context_measure_simple = 'PTS',
              season_nullable = f_season,
              season_type_all_star = 'Regular Season',
            )
  if season == 2003:
    df = json_to_df(shot_json)
  else:
    df = df.append(json_to_df(shot_json))
  season+=1
  time.sleep(10) # Added some delay between API calls just in case.

df.shape

In [ ]:
df = df.reset_index()
df = df.drop(['level_0', 'index'], axis=1)
df.to_csv('/content/drive/MyDrive/Colab_Notebooks/lebron_shots.csv')